In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [7]:
data_dir = 'data'
image_dim = 250
batch_size = 16
steps_per_epoch = 100
num_epochs = 25
classes = ['Lebron_James', 'Stephen_Curry']

In [8]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                             rotation_range=90,
                             width_shift_range=0.1, 
                             height_shift_range=0.1, 
                             shear_range=0.2, 
                             zoom_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=False)

train_generator = datagen.flow_from_directory(os.path.join(data_dir, 'train'),
                                              target_size=(image_dim, image_dim),
                                              color_mode='rgb',
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              classes=classes)

valid_generator = datagen.flow_from_directory(os.path.join(data_dir, 'valid'),
                                              target_size=(image_dim, image_dim),
                                              color_mode='rgb',
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              classes=classes)


Found 673 images belonging to 2 classes.
Found 324 images belonging to 2 classes.


In [9]:
base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(image_dim, image_dim, 3))
base_model.trainable = False

In [10]:
model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(len(classes), activation='softmax')
])

In [11]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

model.fit_generator(train_generator, 
                    steps_per_epoch=steps_per_epoch,
                    epochs=num_epochs,
                    validation_data=valid_generator, 
                    validation_steps=10,
                    verbose=1)

Epoch 1/25
100/100 [==============================] - 139s 1s/step - loss: 0.5132 - accuracy: 0.7567 - val_loss: 0.7684 - val_accuracy: 0.6562
Epoch 2/25
100/100 [==============================] - 107s 1s/step - loss: 0.3978 - accuracy: 0.8264 - val_loss: 1.0276 - val_accuracy: 0.6438
Epoch 3/25
100/100 [==============================] - 119s 1s/step - loss: 0.3063 - accuracy: 0.8713 - val_loss: 0.4603 - val_accuracy: 0.8125
Epoch 4/25
100/100 [==============================] - 146s 1s/step - loss: 0.2887 - accuracy: 0.8778 - val_loss: 0.7356 - val_accuracy: 0.7437
Epoch 5/25
100/100 [==============================] - 104s 1s/step - loss: 0.2634 - accuracy: 0.8892 - val_loss: 0.3875 - val_accuracy: 0.8500
Epoch 6/25
100/100 [==============================] - 117s 1s/step - loss: 0.2757 - accuracy: 0.8968 - val_loss: 0.5523 - val_accuracy: 0.7875
Epoch 7/25
100/100 [==============================] - 132s 1s/step - loss: 0.2537 - accuracy: 0.9000 - val_loss: 0.4130 - val_accuracy: 0.8625

In [28]:
model.save('model.h5')

In [26]:
import cv2
import numpy as np

filename = 'test/lebron.jpg'
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
image = cv2.imread(filename)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
detected_face = face_classifier.detectMultiScale(image, scaleFactor=1.1, minNeighbors=1, minSize=(1,1))
x = detected_face[0][0]
y = detected_face[0][1]
l = detected_face[0][2]
w = detected_face[0][3]
image = image[y:y+l, x:x+w]
image = cv2.resize(image, (image_dim, image_dim))
image = np.float32(image)

In [27]:
model.predict([[image]])

array([[0.9197328, 0.0802672]], dtype=float32)